In [ ]:
import os
from glob import glob
import json
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import time
import wand.image as wi
from PIL import Image
%matplotlib inline
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.__version__)
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

Lets start by defining some functions for handling the images.

In [ ]:
# A function to load the input images and set its dimensions to 1024 x 768
def load_image(image_path):
    max_dim=512
    img = tf.io.read_file(image_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim
    new_shape = tf.cast(shape * scale, tf.int32)

    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]

    return img

def handle_format(image_path):
    img=wi.Image(filename=image_path)
    img.format='jpg'
    new_path = str(image_path).replace('.heic','.jpg')
    img.save(filename=new_path)
    img.close()
    return new_path

def imshow(image, title=None):
    if(len(image.shape) > 3):
        image=np.squeeze(image, axis=0)
    plt.imshow(image)
    if(title):
        plt.title(title)

def my_model(layer_names):
    vgg_model = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
    vgg_model.trainable = False
    outputs = [vgg_model.get_layer(name).output for name in layer_names]
    model=tf.keras.Model([vgg_model.input], outputs)
    return model

def gram_matrix(input_tensor): 
    result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
    input_shape = tf.shape(input_tensor)
    num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32) 
    return result/(num_locations)

def total_cost(outputs):
    style_outputs=outputs['style']
    content_outputs=outputs['content']
    style_loss=tf.add_n([style_weights[name]*tf.reduce_mean((style_outputs[name]-style_targets[name])**2)
                        for name in style_outputs.keys()])
    style_loss*=style_weight/len(style_layers)

    content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-content_targets[name])**2)
                             for name in content_outputs.keys()])
    content_loss*=content_weight/len(content_layers)
    loss=style_loss+content_loss
    return loss

def clip_0_1(image):
    return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

These are Standard Confs for remainder of the work.

In [ ]:
content_layers=['block4_conv2']
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1']

style_weight=30
content_weight=20

style_weights = {'block1_conv1': 0.7,
                 'block2_conv1': 0.2,
                 'block3_conv1': 0.25,
                 'block4_conv1': 0.1,
                 'block5_conv1': 0.25}

In [ ]:
ROOT_DIR = os.path.abspath(os.curdir).split('jupyter_notebooks')[0]

with open(ROOT_DIR+"keys/personal_paths.json", "r") as handler:
    info = json.load(handler)

style_paths = info["style_paths"]
img_paths = info['image_paths']

style_img_list =[]
content_img_list=[]
style_img_name_list= []
content_img_name_list = []

#plt.figure(figsize=(12,12))
for number,filename in enumerate(os.listdir(style_paths[1])):
    style_img=load_image(style_paths[1]+filename)
    style_img_list.append(style_img)
    style_img_name_list.append(filename)

repeat_list_check = []
for number, filename in enumerate(os.listdir(img_paths[1])):
    if filename not in repeat_list_check:
        if '.jpg' not in filename:
            new_filepath = handle_format(img_paths[1]+filename)
            content_img=load_image(new_filepath)
            content_img_name_list.append(new_filepath.replace(img_paths[1],''))
            content_img_list.append(content_img)
            repeat_list_check.append(new_filepath.replace(img_paths[1],''))
        else:
            content_img=load_image(img_paths[1]+filename)
            content_img_name_list.append(filename)
            content_img_list.append(content_img)

In [ ]:
class entire_model(tf.keras.models.Model):
    def __init__(self, style_layers, content_layers):
        super(entire_model, self).__init__()
        self.vgg=my_model(style_layers + content_layers)
        self.style_layers=style_layers
        self.content_layers=content_layers
        self.num_style_layers=len(style_layers)
        self.vgg.trainable=False

    def call(self, inputs):
        inputs=inputs*255.0 
        preprocessed_input=tf.keras.applications.vgg19.preprocess_input(inputs)
        outputs=self.vgg(preprocessed_input)

        style_outputs, content_outputs=(outputs[:self.num_style_layers], outputs[self.num_style_layers:])
        style_outputs=[gram_matrix(layer) for layer in style_outputs]

        content_dict = {content_name:value
                    for content_name, value
                    in zip(self.content_layers, content_outputs)}

        style_dict = {style_name:value
                for style_name, value
                in zip(self.style_layers, style_outputs)}

        return {'content': content_dict, 'style': style_dict}


In [ ]:
for i,content_img in enumerate(content_img_list):
    for j,style_img in enumerate(style_img_list):
        style_name,content_name = style_img_name_list[j],content_img_name_list[i]
        print(style_img_name_list[j])
        print(content_img_name_list[i])

In [ ]:
for i,content_img in enumerate(content_img_list):
    x=tf.keras.applications.vgg19.preprocess_input(content_img*255)
    x=tf.image.resize(x, (256,256))
    content_name = content_img_name_list[i]
    for j,style_img in enumerate(style_img_list):
        style_name = style_img_name_list[j]
        vgg_model=tf.keras.applications.VGG19(include_top=True, weights='imagenet')
        vgg_model.trainable=False

        style_extractor = my_model(style_layers)
        style_outputs = style_extractor(style_img*255)
        
        extractor=entire_model(style_layers, content_layers)
        style_targets = extractor(style_img)['style']
        content_targets = extractor(content_img)['content']

        results = extractor(tf.constant(content_img))

        generate_image = tf.Variable(content_img)
        opt = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)
        
        
        @tf.function()
        def train_step(image):
            with tf.GradientTape() as tape:
                outputs = extractor(image)
                loss = total_cost(outputs)

            grad = tape.gradient(loss, image)
            opt.apply_gradients([(grad, image)])
            image.assign(clip_0_1(image))
            
        num_iterations=2500
        for i in range(num_iterations):
            train_step(generate_image)
            #if(i%500==0):
            #    plt.figure(figsize=(12,12))
            #    plt.subplot(1, 3, 1)
            #    imshow(content_img, 'Original Image')
            #    plt.subplot(1, 3, 2)
            #    imshow(style_img, 'Style Image')
            #    plt.subplot(1, 3, 3)
            #    imshow(np.squeeze(generate_image.read_value(), 0), 'New Image - Step '+str(i))
        train_step(generate_image)
        plt.imshow(np.squeeze(generate_image.read_value(), 0))
        plt.axis('off')
        fig1 = plt.gcf()
        fig1.savefig(f'{ROOT_DIR}results/{content_name}_and_{style_name}.png', bbox_inches='tight')